In [1]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from glob import glob
import os

In [2]:
input_folder = 'test_images/'
output_folder = 'test_images_output/'

In [3]:
# The .prototxt file(s) which define the model architecture (i.e., the layers themselves)
# The .caffemodel file which contains the weights for the actual layers
neural_net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')

In [26]:
def face_detect(image):
    # load the input image and construct an input blob for the image
    # by resizing to a fixed 300x300 pixels and then normalizing it
    # print(image.shape)      # (146, 345, 3)
                              # (168, 300, 3)
                              # (190, 266, 3)
                              # (168, 300, 3)
    height = image.shape[0]
    width = image.shape[1]
    
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    # blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
    # The cv2.dnn.blobFromImage  function returns a blob  which is our input image after mean subtraction, normalizing, and channel swapping.
    # image : This is the input image we want to preprocess before passing it through our deep neural network for classification.
    # scalefactor : After we perform mean subtraction we can optionally scale our images by some factor. This value defaults to `1.0` (i.e., no scaling) but we can supply another value as well.
    # size : Here we supply the spatial size that the Convolutional Neural Network expects.
    # mean : These are our mean subtraction values.
    # swapRB : OpenCV assumes images are in BGR channel order; however, the `mean` value assumes we are using RGB order. 

    # pass the blob through the network and obtain the detections and predictions
    neural_net.setInput(blob)
    detections = neural_net.forward()
    # print(detections.shape)
    # (1, 1, 94, 7)
    # (1, 1, 93, 7)
    # (1, 1, 76, 7)
    # (1, 1, 84, 7)
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is greater than the minimum confidence
        if confidence > 0.80:
            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = box.astype("int")

            # draw the bounding box of the face along with the associated probability
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
            cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            
    return image